In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"
# coding: utf-8
# coding: utf-8

# In[34]:
from datetime import datetime 
import tensorflow as tf
from coco_utils import load_coco_data, decode_captions
from tensorflow.contrib.tensorboard.plugins import projector
from image_utils import image_from_url, write_text_on_image
import numpy as np
import os
import sys

class Config(object):
    def __init__(self):
        
        self.vocab_size =1004
        self.batch_size = 32
        self.initializer_scale =0.08
        self.H = 300 #hidden dimension
        self.T = 16 # caption length
        self.feature_len = 1000
        self.W = 300 # embedding size
        self.num_epochs_per_decay = 1
        self.total_instances = 400135
        self.initial_learning_rate = 2.0
        self.input_len = 16
        self.clip_gradients = 5.0
        self.num_epochs = 10
        self.num_of_layers = 5

def _run_validation(sess, caption_in, caption_out,features, batch_size, model, T):
    """
    Make a single gradient update for batch data. 
    """
    # Make a minibatch of training data
    
    captions_in = caption_in[:, 0].reshape(-1, 1)
    
    state = None 
    final_preds = []
    current_pred = captions_in
    mask = np.zeros((batch_size, 1))
    mask[:, 0] = 1
    
    # get initial state using image feature 
    feed_dict = {model.image_feature: features}
    state = sess.run(model.initial_state, feed_dict=feed_dict)
    
    # start to generate sentences
    val_loss = 0
    for t in range(T):
        
        feed_dict={model.caption_in: current_pred, 
                   model.initial_state : state, 
                   model.caption_mask: mask,
                   model.caption_out: caption_out[:,t].reshape(-1,1)}
        loss = None    
       
        current_pred, state, loss = sess.run([model.preds, model.final_state, model.total_loss], feed_dict=feed_dict)

        #print(loss)
        val_loss += loss
        current_pred = current_pred.reshape(-1, 1)   
        final_preds.append(current_pred)
        
    val_loss /= T
   
    return final_preds, val_loss

def minibatch(data, index, batch_size,total_size, split='train'):
    #batch_size = batch_size+1
    begin = batch_size*index%total_size
    end = begin+ batch_size
    if end > total_size:
        #print(begin)
        end = end - total_size# minus sign
        caption_end = data['%s_captions'%split][begin:]
        caption_first = data['%s_captions'%split][:end]
        image_idxs_end = data['%s_image_idxs'%split][:end]
        image_idxs_first = data['%s_image_idxs'%split][begin:]
        image_idxs = np.append(image_idxs_end, image_idxs_first, axis=0)
        caption = np.append(caption_end, caption_first,axis=0)
    else:
        caption = data['%s_captions'%split][begin:end]
        image_idxs = data['%s_image_idxs'%split][begin:end]
    #print(image_idxs)
    image_features = data['%s_features' % split][image_idxs]
    urls = data['%s_urls' % split][image_idxs]
    caption_in = caption[:,:-1]
    caption_out = caption[:,1:]
    mask = (caption_out != 0)
    
    return caption_in, caption_out, mask, image_features, urls

        
class LSTM_Model:
    def __init__(self, mode, config):
        self.config = config
        self.initializer = tf.random_uniform_initializer(
            minval=-self.config.initializer_scale,
            maxval=self.config.initializer_scale)
        self.global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name="global_step")        
        #Mode can be "train", "test", "infer"
        #assert error if the mode is not one of the three predesigned modes.
#         assert mode in ['train','test','infer']
#     def is_training(self):
#         return self.mode == "train"
  
    def _build_embedding(self):
        import pickle
        with tf.variable_scope("word_embedding"):
            self.caption_in = tf.placeholder(tf.int32,[self.config.batch_size, None], name="caption_in")
            self.embed_map = tf.get_variable(name="embed_map", 
                                           shape=[self.config.vocab_size, self.config.W],
                                           initializer = self.initializer)
            word_vectors = tf.nn.embedding_lookup(self.embed_map, self.caption_in)
        self.word_embedding = word_vectors
        
        with tf.variable_scope("image_embedding"):
            self.image_feature = tf.placeholder(tf.float32,[self.config.batch_size, self.config.feature_len], name="image_feature")
            feature_embedding = tf.contrib.layers.fully_connected(
            inputs=self.image_feature,
            num_outputs= self.config.H,
            activation_fn=None,
            weights_initializer= self.initializer,
            biases_initializer=None)
        
        self.feature_embedding = feature_embedding
    
    def _build_model(self):
        #self.mode = tf.placeholder(tf.string, name='mode')
        lstm_cell = tf.contrib.rnn.BasicLSTMCell(
            num_units = self.config.H, state_is_tuple =True)
        lstm_cell = tf.contrib.rnn.MultiRNNCell([lstm_cell] * self.config.num_of_layers)

        # drop out is not included
        with tf.variable_scope("lstm", initializer = self.initializer) as lstm_scope:
            self.caption_out = tf.placeholder(tf.int32,[self.config.batch_size, None], name="caption_out")
            self.caption_mask = tf.placeholder(tf.int32,[self.config.batch_size, None], name="caption_mask")
            zero_state = lstm_cell.zero_state(
                batch_size=self.config.batch_size,dtype=tf.float32)
            _, self.initial_state = lstm_cell(self.feature_embedding, zero_state)

            lstm_scope.reuse_variables()
            sequence_len = tf.reduce_sum(self.caption_mask,1)
            lstm_out, self.final_state = tf.nn.dynamic_rnn(cell=lstm_cell,
                                              inputs = self.word_embedding,
                                              sequence_length = sequence_len,
                                              initial_state = self.initial_state,
                                              dtype = tf.float32,
                                              scope = lstm_scope)
        #to stack batches vertically
        lstm_out = tf.reshape(lstm_out, [-1, lstm_cell.output_size])

        with tf.variable_scope("logits") as logits_scope:
            #w = tf.get_variable('w', [lstm_cell.output_size, self.config.vocab_size], initializer=self.initializer)
            #b = tf.get_variable('b', [self.config.vocab_size], initializer=tf.constant_initializer(0.0))
            # (Nt)*H ,H*v =Nt.V, bias is zero
            #tf.summary.histogram("weights", w)
            #logits = tf.matmul(lstm_out,w)+b
            #variable_summaries(w)
            logits = tf.contrib.layers.fully_connected(
                inputs = lstm_out,
                num_outputs = self.config.vocab_size,
                activation_fn = None,
                weights_initializer = self.initializer,
                scope = logits_scope
            )
            #print(logits.get_shape())

        with tf.variable_scope("loss"):
            #if self.mode == 'train':
            targets = tf.reshape(self.caption_out,[-1])
            mask = tf.to_float(tf.reshape(self.caption_mask,[-1]))
            loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=targets,
                                                     logits = logits)
            #loss = tf.reshape(loss,[loss.get_shape()[0],1])
            #loss = np.reshape(loss, (loss.shape[0],1))
            #print(loss.get_shape())
            batch_loss = tf.div(tf.reduce_sum(tf.multiply(loss, mask)),
                                tf.reduce_sum(mask),
                                name="batch_loss")
            tf.losses.add_loss(batch_loss)
            self.total_loss = tf.losses.get_total_loss()
            #print(self.total_loss.get_shape())
            #if self.mode == 'evaluation':
            self.preds = tf.argmax(logits, 1)
            
    def _create_summaries(self):
        with tf.name_scope("summaries"):
            tf.summary.scalar("train-loss", self.total_loss)
            tf.summary.histogram("histogramforloss", self.total_loss)
            # because you have several summaries, we should merge them all
            # into one op to make it easier to manage
            self.summary_op = tf.summary.merge_all()
            

    def build_graph(self):
        #tf.reset_default_graph()
        self._build_embedding()
        self._build_model()
        self._create_summaries()

def train_model(model, config, data):
    
    #g = tf.Graph()
    #with g.as_default():
    ################define optimizer########
    num_batches = config.total_instances/config.batch_size
    decay_steps = int(num_batches*config.num_epochs_per_decay)
    learning_rate = tf.constant(config.initial_learning_rate)

    learning_rate_decay_fn = None
    def _decay_fn(learning_rate, global_step):
        return tf.train.exponential_decay(learning_rate,
                                         global_step,
                                         decay_steps = decay_steps,
                                         decay_rate=0.5,
                                         staircase=True)

    learning_rate_decay_fn = _decay_fn
    train_op = tf.contrib.layers.optimize_loss(loss=model.total_loss,
                                              global_step = model.global_step,
                                              learning_rate = learning_rate,
                                              optimizer = 'SGD',
                                              clip_gradients = config.clip_gradients,
                                              learning_rate_decay_fn =learning_rate_decay_fn)

    ##################
    
    saver = tf.train.Saver()
    init = tf.global_variables_initializer()
    val_losses = [];
    #sum_op = tf.summary.scalar("val-loss", val_losss)
    #print(sum_op)
    ##get the word embedding from the file
    import pickle
    with open('embedding.txt','rb') as fb:
        value = pickle.load(fb)
    with tf.Session() as sess:
        sess.run(init)
        # if checkpoint exist, restore
        #ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/checkpoint'))
        #if ckpt and ckpt.model_checkpoint_path:
        #    saver.restore(sess, ckpt.model_checkpoint_path)
        sess.run(model.embed_map.assign(value[:-1]))

        
        # 100 epoch
        total_runs = int((config.total_instances/config.batch_size)*config.num_epochs)
        initial_step = model.global_step.eval()
        
        ### initialize summary writer
        learning_change = tf.summary.scalar("learing_rate", learning_rate)
        #a = tf.summary.merge_all()
        
        writer = tf.summary.FileWriter('./graphs/singlelayer_lstm_glove_embedding_alexnet_5', sess.graph)
        #writer_tzr_handsome = tf.summary.FileWriter('./graphs/eval/singlelayer_lstm_eval', sess.graph)
        time_now = datetime.now()
        for t in range(total_runs):

            caption_in, caption_out, mask, image_features, urls = minibatch(data,t,config.batch_size, config.total_instances)
            
            # feed data
            feed_dict = {model.image_feature: image_features, model.caption_in: caption_in, 
                        model.caption_out: caption_out, model.caption_mask: mask}
            merge_op, _, total_loss, learning = sess.run([model.summary_op, train_op, model.total_loss, learning_change],
                                           feed_dict = feed_dict)
            #print("total loss")
            #print(total_loss)
            writer.add_summary(merge_op, global_step=t)
            writer.add_summary(learning, global_step=t)
            
            # print loss infor
            if(t+1) % 20 == 0:
                print('(Iteration %d / %d) loss: %f, and time eclipsed: %.2f minutes' % (
                    t + 1, total_runs, float(total_loss), (datetime.now() - time_now).seconds/60.0))
            
            #validation loss and generate captions
            #if(t+1)%10 == 0 or t ==(total_runs-1):
            if not os.path.exists('test_caption'):
                os.makedirs('test_caption')
            captions_pred, val_loss = _run_validation(sess, caption_in, caption_out, image_features, config.batch_size, model, config.input_len) # the output is size (32, 16)
            captions_pred = [unpack.reshape(-1, 1) for unpack in captions_pred]
            captions_pred = np.concatenate(captions_pred, 1)
            captions_deco = decode_captions(captions_pred, data['idx_to_word'])
#                 for j in range(len(captions_deco)):
#                     img_name = os.path.join('test_caption', 'image_{}.jpg'.format(j))
#                     img = image_from_url(urls[j])
#                     write_text_on_image(img, img_name, captions_deco[j])
            
    
            #print(sum_op)
            #add_val = sess.run(sum_op)
            #writer.add_summary(add_val, global_step = t)
            with open("val_loss_alex.txt", "a") as myfile:
                myfile.write(str(val_loss)+"\n")
               

            #save model
            if(t+1)%50 == 0 or t == (total_runs-1):
                if not os.path.exists('checkpoints/singlelayer_lstm_glove_embedding_alexnet_5'):
                    os.makedirs('checkpoints/singlelayer_lstm_glove_embedding_alexnet_5')
                saver.save(sess, 'checkpoints/singlelayer_lstm_glove_embedding_alexnet_5/singlelayer_lstm_glove_embedding_alexnet', t)
        
        # visualize embed matrix
        #code to visualize the embeddings. uncomment the below to visualize embeddings
        final_embed_matrix = sess.run(model.embed_map)
        
        # it has to variable. constants don't work here. you can't reuse model.embed_matrix
        embedding_var = tf.Variable(final_embed_matrix[:1000], name='embedding')
        sess.run(embedding_var.initializer)

        config = projector.ProjectorConfig()
        summary_writer = tf.summary.FileWriter('processed')

        # add embedding to the config file
        embedding = config.embeddings.add()
        embedding.tensor_name = embedding_var.name
        
        # link this tensor to its metadata file, in this case the first 500 words of vocab
#         metadata_path = './processed/matadata.tsv'
#         if not os.path.exists(metadata_path):
#             f = open(metadata_path, "w")
#             f.close()
        embedding.metadata_path = os.path.join('processed', 'metadata.tsv')

        # saves a configuration file that TensorBoard will read during startup.
        projector.visualize_embeddings(summary_writer, config)
        saver_embed = tf.train.Saver([embedding_var])
        saver_embed.save(sess, 'processed/model3.ckpt', 1)


def main():
    config = Config()
    data = load_coco_data()
    model = LSTM_Model('train', config)
    model.build_graph()
    train_model(model, config, data)

main()

(Iteration 20 / 125042) loss: 6.475111, and time eclipsed: 0.05 minutes
(Iteration 40 / 125042) loss: 4.799927, and time eclipsed: 0.10 minutes
(Iteration 60 / 125042) loss: 4.570457, and time eclipsed: 0.17 minutes
(Iteration 80 / 125042) loss: 4.595694, and time eclipsed: 0.22 minutes
(Iteration 100 / 125042) loss: 4.416484, and time eclipsed: 0.27 minutes
(Iteration 120 / 125042) loss: 4.369204, and time eclipsed: 0.35 minutes
(Iteration 140 / 125042) loss: 4.234069, and time eclipsed: 0.38 minutes
(Iteration 160 / 125042) loss: 4.086136, and time eclipsed: 0.47 minutes
(Iteration 180 / 125042) loss: 4.122127, and time eclipsed: 0.52 minutes
(Iteration 200 / 125042) loss: 4.218237, and time eclipsed: 0.57 minutes
(Iteration 220 / 125042) loss: 4.450221, and time eclipsed: 0.63 minutes
(Iteration 240 / 125042) loss: 3.940015, and time eclipsed: 0.68 minutes
(Iteration 260 / 125042) loss: 3.967720, and time eclipsed: 0.77 minutes
(Iteration 280 / 125042) loss: 4.130177, and time eclip

(Iteration 2260 / 125042) loss: 3.497231, and time eclipsed: 6.53 minutes
(Iteration 2280 / 125042) loss: 3.524622, and time eclipsed: 6.58 minutes
(Iteration 2300 / 125042) loss: 3.149454, and time eclipsed: 6.63 minutes
(Iteration 2320 / 125042) loss: 3.289227, and time eclipsed: 6.70 minutes
(Iteration 2340 / 125042) loss: 3.296701, and time eclipsed: 6.75 minutes
(Iteration 2360 / 125042) loss: 3.068803, and time eclipsed: 6.83 minutes
(Iteration 2380 / 125042) loss: 3.200996, and time eclipsed: 6.87 minutes
(Iteration 2400 / 125042) loss: 2.701056, and time eclipsed: 6.93 minutes
(Iteration 2420 / 125042) loss: 3.342594, and time eclipsed: 7.00 minutes
(Iteration 2440 / 125042) loss: 3.331457, and time eclipsed: 7.05 minutes
(Iteration 2460 / 125042) loss: 3.007862, and time eclipsed: 7.12 minutes
(Iteration 2480 / 125042) loss: 3.845188, and time eclipsed: 7.17 minutes
(Iteration 2500 / 125042) loss: 3.198738, and time eclipsed: 7.22 minutes
(Iteration 2520 / 125042) loss: 3.1000

(Iteration 4480 / 125042) loss: 3.128470, and time eclipsed: 12.95 minutes
(Iteration 4500 / 125042) loss: 2.387204, and time eclipsed: 13.00 minutes
(Iteration 4520 / 125042) loss: 2.482722, and time eclipsed: 13.07 minutes
(Iteration 4540 / 125042) loss: 2.818557, and time eclipsed: 13.12 minutes
(Iteration 4560 / 125042) loss: 2.595438, and time eclipsed: 13.18 minutes
(Iteration 4580 / 125042) loss: 2.885381, and time eclipsed: 13.23 minutes
(Iteration 4600 / 125042) loss: 2.922384, and time eclipsed: 13.28 minutes
(Iteration 4620 / 125042) loss: 2.640319, and time eclipsed: 13.35 minutes
(Iteration 4640 / 125042) loss: 2.751047, and time eclipsed: 13.40 minutes
(Iteration 4660 / 125042) loss: 2.771521, and time eclipsed: 13.47 minutes
(Iteration 4680 / 125042) loss: 2.605879, and time eclipsed: 13.52 minutes
(Iteration 4700 / 125042) loss: 2.721608, and time eclipsed: 13.57 minutes
(Iteration 4720 / 125042) loss: 2.441047, and time eclipsed: 13.63 minutes
(Iteration 4740 / 125042)

(Iteration 6680 / 125042) loss: 2.690882, and time eclipsed: 19.22 minutes
(Iteration 6700 / 125042) loss: 2.753138, and time eclipsed: 19.27 minutes
(Iteration 6720 / 125042) loss: 2.209826, and time eclipsed: 19.33 minutes
(Iteration 6740 / 125042) loss: 2.360674, and time eclipsed: 19.38 minutes
(Iteration 6760 / 125042) loss: 2.642847, and time eclipsed: 19.47 minutes
(Iteration 6780 / 125042) loss: 2.823634, and time eclipsed: 19.50 minutes
(Iteration 6800 / 125042) loss: 2.279808, and time eclipsed: 19.55 minutes
(Iteration 6820 / 125042) loss: 2.432527, and time eclipsed: 19.62 minutes
(Iteration 6840 / 125042) loss: 2.105820, and time eclipsed: 19.67 minutes
(Iteration 6860 / 125042) loss: 2.331036, and time eclipsed: 19.75 minutes
(Iteration 6880 / 125042) loss: 2.595149, and time eclipsed: 19.80 minutes
(Iteration 6900 / 125042) loss: 2.353433, and time eclipsed: 19.83 minutes
(Iteration 6920 / 125042) loss: 2.215221, and time eclipsed: 19.92 minutes
(Iteration 6940 / 125042)

(Iteration 8880 / 125042) loss: 2.195153, and time eclipsed: 25.48 minutes
(Iteration 8900 / 125042) loss: 2.428421, and time eclipsed: 25.52 minutes
(Iteration 8920 / 125042) loss: 2.350359, and time eclipsed: 25.58 minutes
(Iteration 8940 / 125042) loss: 2.488219, and time eclipsed: 25.62 minutes
(Iteration 8960 / 125042) loss: 2.328321, and time eclipsed: 25.68 minutes
(Iteration 8980 / 125042) loss: 2.717304, and time eclipsed: 25.73 minutes
(Iteration 9000 / 125042) loss: 2.429315, and time eclipsed: 25.78 minutes
(Iteration 9020 / 125042) loss: 2.431200, and time eclipsed: 25.85 minutes
(Iteration 9040 / 125042) loss: 2.340850, and time eclipsed: 25.90 minutes
(Iteration 9060 / 125042) loss: 2.650082, and time eclipsed: 25.97 minutes
(Iteration 9080 / 125042) loss: 2.282424, and time eclipsed: 26.02 minutes
(Iteration 9100 / 125042) loss: 2.373221, and time eclipsed: 26.07 minutes
(Iteration 9120 / 125042) loss: 2.158081, and time eclipsed: 26.13 minutes
(Iteration 9140 / 125042)

...

(Iteration 116000 / 125042) loss: 2.290312, and time eclipsed: 283.57 minutes
(Iteration 116020 / 125042) loss: 1.999320, and time eclipsed: 283.63 minutes
(Iteration 116040 / 125042) loss: 2.536562, and time eclipsed: 283.68 minutes
(Iteration 116060 / 125042) loss: 2.175051, and time eclipsed: 283.75 minutes
(Iteration 116080 / 125042) loss: 1.968519, and time eclipsed: 283.80 minutes
(Iteration 116100 / 125042) loss: 2.243598, and time eclipsed: 283.85 minutes
(Iteration 116120 / 125042) loss: 2.247406, and time eclipsed: 283.92 minutes
(Iteration 116140 / 125042) loss: 2.291865, and time eclipsed: 283.97 minutes
(Iteration 116160 / 125042) loss: 2.282611, and time eclipsed: 284.03 minutes
(Iteration 116180 / 125042) loss: 2.423012, and time eclipsed: 284.08 minutes
(Iteration 116200 / 125042) loss: 2.450471, and time eclipsed: 284.13 minutes
(Iteration 116220 / 125042) loss: 2.119237, and time eclipsed: 284.20 minutes
(Iteration 116240 / 125042) loss: 2.121989, and time eclipsed: 2

(Iteration 118120 / 125042) loss: 2.218036, and time eclipsed: 289.58 minutes
(Iteration 118140 / 125042) loss: 2.292168, and time eclipsed: 289.63 minutes
(Iteration 118160 / 125042) loss: 2.343661, and time eclipsed: 289.70 minutes
(Iteration 118180 / 125042) loss: 2.177456, and time eclipsed: 289.75 minutes
(Iteration 118200 / 125042) loss: 2.242541, and time eclipsed: 289.80 minutes
(Iteration 118220 / 125042) loss: 2.527959, and time eclipsed: 289.87 minutes
(Iteration 118240 / 125042) loss: 2.370580, and time eclipsed: 289.92 minutes
(Iteration 118260 / 125042) loss: 2.072512, and time eclipsed: 289.98 minutes
(Iteration 118280 / 125042) loss: 2.421138, and time eclipsed: 290.03 minutes
(Iteration 118300 / 125042) loss: 2.339667, and time eclipsed: 290.08 minutes
(Iteration 118320 / 125042) loss: 2.160050, and time eclipsed: 290.15 minutes
(Iteration 118340 / 125042) loss: 2.224857, and time eclipsed: 290.20 minutes
(Iteration 118360 / 125042) loss: 2.538328, and time eclipsed: 2

(Iteration 120240 / 125042) loss: 2.034150, and time eclipsed: 295.60 minutes
(Iteration 120260 / 125042) loss: 2.107619, and time eclipsed: 295.67 minutes
(Iteration 120280 / 125042) loss: 2.238446, and time eclipsed: 295.72 minutes
(Iteration 120300 / 125042) loss: 1.984278, and time eclipsed: 295.75 minutes
(Iteration 120320 / 125042) loss: 2.236214, and time eclipsed: 295.83 minutes
(Iteration 120340 / 125042) loss: 2.031656, and time eclipsed: 295.88 minutes
(Iteration 120360 / 125042) loss: 1.981021, and time eclipsed: 295.93 minutes
(Iteration 120380 / 125042) loss: 2.338661, and time eclipsed: 295.98 minutes
(Iteration 120400 / 125042) loss: 2.436141, and time eclipsed: 296.03 minutes
(Iteration 120420 / 125042) loss: 2.254219, and time eclipsed: 296.10 minutes
(Iteration 120440 / 125042) loss: 1.985723, and time eclipsed: 296.15 minutes
(Iteration 120460 / 125042) loss: 2.214865, and time eclipsed: 296.22 minutes
(Iteration 120480 / 125042) loss: 2.216296, and time eclipsed: 2

(Iteration 122360 / 125042) loss: 1.904431, and time eclipsed: 301.58 minutes
(Iteration 122380 / 125042) loss: 2.010859, and time eclipsed: 301.63 minutes
(Iteration 122400 / 125042) loss: 1.891524, and time eclipsed: 301.68 minutes
(Iteration 122420 / 125042) loss: 2.000504, and time eclipsed: 301.75 minutes
(Iteration 122440 / 125042) loss: 1.921465, and time eclipsed: 301.80 minutes
(Iteration 122460 / 125042) loss: 2.557567, and time eclipsed: 301.87 minutes
(Iteration 122480 / 125042) loss: 2.014802, and time eclipsed: 301.92 minutes
(Iteration 122500 / 125042) loss: 2.203083, and time eclipsed: 301.95 minutes
(Iteration 122520 / 125042) loss: 2.205572, and time eclipsed: 302.02 minutes
(Iteration 122540 / 125042) loss: 2.095563, and time eclipsed: 302.07 minutes
(Iteration 122560 / 125042) loss: 2.185073, and time eclipsed: 302.13 minutes
(Iteration 122580 / 125042) loss: 2.074801, and time eclipsed: 302.18 minutes
(Iteration 122600 / 125042) loss: 2.037974, and time eclipsed: 3

(Iteration 124480 / 125042) loss: 2.274309, and time eclipsed: 307.53 minutes
(Iteration 124500 / 125042) loss: 2.342956, and time eclipsed: 307.57 minutes
(Iteration 124520 / 125042) loss: 2.406164, and time eclipsed: 307.63 minutes
(Iteration 124540 / 125042) loss: 2.240183, and time eclipsed: 307.68 minutes
(Iteration 124560 / 125042) loss: 2.162004, and time eclipsed: 307.75 minutes
(Iteration 124580 / 125042) loss: 2.135174, and time eclipsed: 307.80 minutes
(Iteration 124600 / 125042) loss: 2.405786, and time eclipsed: 307.85 minutes
(Iteration 124620 / 125042) loss: 1.985101, and time eclipsed: 307.92 minutes
(Iteration 124640 / 125042) loss: 2.208246, and time eclipsed: 307.97 minutes
(Iteration 124660 / 125042) loss: 2.361624, and time eclipsed: 308.03 minutes
(Iteration 124680 / 125042) loss: 1.968768, and time eclipsed: 308.08 minutes
(Iteration 124700 / 125042) loss: 2.114575, and time eclipsed: 308.13 minutes
(Iteration 124720 / 125042) loss: 2.210618, and time eclipsed: 3